# stormchasers - Extended Feature Selection
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Feb 22, 2022

## Hypothesis
Two very important components of a machine learning model are feature selection and feature engineering. Our idea is that adding some more features to the StellarAlgo retention model could improve performance of the model. This notebook will test the standard StellarAlgo retention model features.

## Experiment
This section details our experiment including querying data, data transformations, feature selection and modelling.

In [1]:
import getpass
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from pycaret.classification import *

warnings.filterwarnings('ignore')

### Let's connect to MSSQL and run a stored proc to get our dataset:

In [2]:
# connect to SQL Server.
SERVER = '34.206.73.189' 
DATABASE = 'datascience' 
USERNAME = 'nrad' 
PASSWORD = getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

In [3]:
lkupclientid = 43 # stormchasers

cursor = CNXN.cursor()

storedProc = (
    f"""Exec [stlrMILB].[ds].[getRetentionScoringModelData] {lkupclientid}"""
)

df = pd.read_sql(storedProc, CNXN)

# apply some data transformations
df["year"] = pd.to_numeric(df["year"])

CNXN.commit()
cursor.close()

df.head()
#df.info()

,lkupClientId,dimCustomerMasterId,customerNumber,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,source_tenure,tenure,distToVenue,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,click_link,fill_out_form,open_email,send_email,unsubscribe_email,openToSendRatio,clickToSendRatio,clickToOpenRatio,posting_records,resale_records,resale_atp,forward_records,cancel_records,email,inbound_email,inbound_phonecall,inperson_contact,internal_note,left_message,outbound_email,outbound_phonecall,phonecall,text,unknown,gender,childrenPresentInHH,maritalStatus,lengthOfResidenceInYrs,annualHHIncome,education,urbanicity,credits_after_refund,is_Lockdown,NumberofGamesPerSeason,CNTPostponedGames,isNextYear_Buyer
0,43,321077826,323791,2017,Full Season,2160.0,2017-09-04,0.326389,50,TRUE,1095,196,6.09,33,0,0,1,7,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,1
1,43,321077826,323791,2017,Partial Season,0.0,2017-08-04,1.000000,0,FALSE,1095,165,6.09,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,1
2,43,351596640,319433,2017,Full Season,5284.0,2017-09-02,0.303571,151,TRUE,1095,297,2.39,25,1,3,2,8,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,1
3,43,351964138,319377,2017,Full Season,4204.0,2017-09-03,0.246479,76,TRUE,1095,222,2.39,22,0,1,1,6,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,1
4,43,353230755,9255400,2017,Partial Season,596.0,2017-09-03,0.875000,38,TRUE,178,178,9.17,8,0,3,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2533 entries, 0 to 2532
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lkupClientId            2533 non-null   int64  
 1   dimCustomerMasterId     2533 non-null   int64  
 2   customerNumber          2533 non-null   object 
 3   year                    2533 non-null   int64  
 4   productGrouping         2533 non-null   object 
 5   totalSpent              2533 non-null   float64
 6   recentDate              2533 non-null   object 
 7   attendancePercent       2533 non-null   float64
 8   renewedBeforeDays       2533 non-null   int64  
 9   isBuyer                 2533 non-null   object 
 10  source_tenure           2533 non-null   int64  
 11  tenure                  2533 non-null   int64  
 12  distToVenue             2533 non-null   float64
 13  totalGames              2533 non-null   int64  
 14  recency                 2533 non-null   

### Let's also drop the features that only have a single value, as they won't add much differentiation to our model:

In [5]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)
        
df.shape

(2533, 20)

### We should also drop features that have a low correlation with the target label as they won't be useful for prediction, we'll only keep features that have a correlation above a set threshold:

In [6]:
cor = df.corr()

threshold = 0.05

#Correlation with output variable
cor_target = abs(cor["isNextYear_Buyer"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target > threshold]

feats = []
for name, val in relevant_features.items():
    feats.append(name)

df_correlated = df[feats]

df_correlated.shape

df_correlated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2533 entries, 0 to 2532
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   dimCustomerMasterId     2533 non-null   int64  
 1   year                    2533 non-null   int64  
 2   totalSpent              2533 non-null   float64
 3   attendancePercent       2533 non-null   float64
 4   renewedBeforeDays       2533 non-null   int64  
 5   source_tenure           2533 non-null   int64  
 6   totalGames              2533 non-null   int64  
 7   missed_games_1          2533 non-null   int64  
 8   missed_games_2          2533 non-null   int64  
 9   missed_games_over_2     2533 non-null   int64  
 10  is_Lockdown             2533 non-null   int64  
 11  NumberofGamesPerSeason  2533 non-null   int64  
 12  isNextYear_Buyer        2533 non-null   int64  
dtypes: float64(2), int64(11)
memory usage: 257.4 KB


### Now that we have the right features we can look at the correlations between them, if features are highly correlated with each other it might negatively impact the model:

In [7]:
corr = df_correlated.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,dimCustomerMasterId,year,totalSpent,attendancePercent,renewedBeforeDays,source_tenure,totalGames,missed_games_1,missed_games_2,missed_games_over_2,is_Lockdown,NumberofGamesPerSeason,isNextYear_Buyer
dimCustomerMasterId,1.00,-0.05,-0.03,0.02,-0.01,0.04,-0.04,-0.03,-0.04,-0.07,-0.05,-0.04,-0.05
year,-0.05,1.00,0.10,-0.14,-0.13,0.06,0.09,-0.00,-0.01,0.08,0.84,0.78,-0.13
totalSpent,-0.03,0.10,1.00,-0.30,0.14,0.02,0.53,0.08,0.17,0.47,0.06,0.12,0.26
attendancePercent,0.02,-0.14,-0.30,1.00,-0.10,-0.08,-0.02,0.04,-0.12,-0.50,-0.12,-0.14,-0.11
renewedBeforeDays,-0.01,-0.13,0.14,-0.10,1.00,0.04,0.32,0.26,0.23,0.24,-0.30,-0.05,0.28
source_tenure,0.04,0.06,0.02,-0.08,0.04,1.00,0.11,0.09,0.08,0.09,0.02,0.08,0.13
totalGames,-0.04,0.09,0.53,-0.02,0.32,0.11,1.00,0.61,0.52,0.52,0.07,0.10,0.46
missed_games_1,-0.03,-0.00,0.08,0.04,0.26,0.09,0.61,1.00,0.52,0.27,-0.01,0.03,0.29
missed_games_2,-0.04,-0.01,0.17,-0.12,0.23,0.08,0.52,0.52,1.00,0.41,-0.01,0.01,0.28
missed_games_over_2,-0.07,0.08,0.47,-0.50,0.24,0.09,0.52,0.27,0.41,1.00,0.05,0.10,0.38


### In order to compare two sets of features, we need to create some datasets for training and evalution:


In [8]:
# select % of the data for training
df_train = df_correlated.sample(frac=0.8, random_state=786).reset_index(drop=True)

# create the eval datasets for A and B
df_eval = df_correlated.drop(df_train.index).reset_index(drop=True)

# print out the number of records for training and eval
print('Data for Modeling: ' + str(df_train.shape))
print('Unseen Data For Predictions: ' + str(df_eval.shape), end="\n\n")

Data for Modeling: (2026, 13)
Unseen Data For Predictions: (507, 13)



### Now we can model the data using a binary classification prediction for the isnextyear_buyer field to see how likely a customer is to re-purchase.

In [9]:
setup(
    data= df_train, 
    target="isNextYear_Buyer", 
    train_size = 0.80,
    data_split_shuffle=True,
    silent=True,
    numeric_features=[
        "totalSpent",
        "attendancePercent",
        "renewedBeforeDays",
        "source_tenure",
        "totalGames",
        "missed_games_1",
        "missed_games_2",
        "missed_games_over_2",
        "is_Lockdown",
        "NumberofGamesPerSeason"
    ]
)

,Description,Value
0,session_id,444
1,Target,isNextYear_Buyer
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(2026, 13)"
5,Missing Values,False
6,Numeric Features,11
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


([],
 'clf-default-name',
       dimCustomerMasterId  year  totalSpent  attendancePercent  \
 0               457223783  2018      6390.0           0.282407   
 1               458839924  2017         0.0           0.888889   
 2               457208926  2017         0.0           1.000000   
 3               457224761  2021       297.0           0.592593   
 4               457224037  2018      4828.0           0.317726   
 ...                   ...   ...         ...                ...   
 2021            457225252  2018       240.0           0.750000   
 2022            457208744  2019      2265.0           0.776224   
 2023            457209244  2017      4320.0           0.176056   
 2024            457212283  2017      5284.0           0.484536   
 2025            457219200  2019       236.0           1.000000   
 
       renewedBeforeDays  source_tenure  totalGames  missed_games_1  \
 0                   155           1095          27               3   
 1                     1  

In [10]:
model_matrix = compare_models(
    fold=10,
    include=["lr", "xgboost"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7556,0.8374,0.7941,0.7545,0.7732,0.5085,0.5104,0.3210
lr,Logistic Regression,0.5247,0.4687,1.0000,0.5247,0.6883,0.0000,0.0000,0.2920


In [11]:
best_model = create_model(model_matrix)
final_model = finalize_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7469,0.8544,0.7647,0.7558,0.7602,0.4923,0.4923
1,0.7284,0.7986,0.7765,0.7253,0.7500,0.4535,0.4547
2,0.7160,0.8150,0.7529,0.7191,0.7356,0.4293,0.4298
3,0.7593,0.7997,0.8588,0.7300,0.7892,0.5129,0.5221
4,0.8148,0.8866,0.8118,0.8313,0.8214,0.6292,0.6294
5,0.7346,0.8523,0.7882,0.7283,0.7571,0.4656,0.4673
6,0.7716,0.8671,0.8353,0.7553,0.7933,0.5396,0.5430
7,0.7778,0.8533,0.7765,0.7952,0.7857,0.5550,0.5552
8,0.7716,0.8220,0.8235,0.7609,0.7910,0.5401,0.5422
9,0.7346,0.8252,0.7529,0.7442,0.7485,0.4675,0.4675


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

## Observations
Here you can document some ideas on the results from above

## Conclusions
Here you can talk about next steps, did the experiment work? If yes, what to do next? If no, why?